In [18]:
import xml.etree.ElementTree as ET
from pathlib import Path
import numpy as np
from enum import Enum
from collections import namedtuple

In [2]:
Mesh = namedtuple('Mesh', ['mesh_pos', 'connectivity', 'scalar_field'])

In [3]:
dtypes = {
    "Int8"   : int,
    "UInt8"  : int,
    "Int16"  : int,
    "UInt16" : int,
    "Int32"  : int,
    "UInt32" : int,
    "Int64"  : int,
    "UInt64" : int,
    "Float32": float,
    "Float64": float}

In [4]:
path_to_file = Path("solution.pvd")
# parsing solution file
tree = ET.parse(path_to_file)
root = tree.getroot()
ds_path = None
for ds in root.iter("DataSet"):
    ds_path = Path(ds.get("file"))

In [5]:
def parse_string(x, dtype, noc=3):
    arr = np.fromstring(x, sep=" ", dtype=dtype)
    return arr.reshape(-1, noc)

In [9]:
def parse_solution(ds_path):
    
    mesh_pos = None
    connectivity = None
    scalar_field = None
    
    tree = ET.parse(ds_path)
    root = tree.getroot()

    # Points
    for points in root.iter("Points"):
        dataArray = points.find("DataArray")
        mesh_pos = parse_string(dataArray.text, dtype=dtypes[dataArray.attrib["type"]], noc=3)
    # Cells
    for cells in root.iter("Cells"):
        for dataArray in cells.iter("DataArray"):
            if dataArray.attrib["Name"] == "connectivity":
                connectivity = parse_string(dataArray.text, dtype=dtypes[dataArray.attrib["type"]], noc=3)
    # Fields
    for pointData in root.iter("PointData"):
        for dataArray in pointData.iter("DataArray"):
            scalar_field = parse_string(dataArray.text, dtype=dtypes[dataArray.attrib["type"]], noc=1)
        
    return Mesh(mesh_pos, connectivity, scalar_field)

In [10]:
mesh_1 = parse_solution(ds_path)